# Deep Convolutional Generative Adversarial Networks (DCGAN)

The following DC-GAN training code is written to be executed via Google Colab using GDrive as the file storage.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Insert the directory
import sys
sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks')

In [3]:
# Load all relevant modules
import os
import time as timer

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

import numpy as np
from IPython import display
from skimage.util import random_noise

from torchvision import datasets
import torchvision.transforms as T
import torchvision.utils as vutils

import res.viz_utils as vu
from res.plot_lib import set_default

set_default()

In [4]:
GDRIVE_DIR = "/content/drive/My Drive/Colab Notebooks/"

# Define default constants

# Data related constants
# DATASET = 'cifar10'
# DATASET = 'cifar100'
# DATASET = 'celebA'
DATASET = 'svhn'

DATA_DIR = os.path.join(GDRIVE_DIR, "data")
BATCH_SIZE = 64
NUM_EPOCH = 25
IMAGE_SIZE = 64
NUM_WORKERS = 2
DAY = '4may'

# Model related constants
MODEL_DIR = os.path.join(GDRIVE_DIR, "models")
SAMPLE_DIR = os.path.join(MODEL_DIR, f"gen_samples_{DATASET}_{DAY}")

# create SAMPLE_DIR if not exists
if not os.path.exists(SAMPLE_DIR):
    os.makedirs(SAMPLE_DIR)
    print(f'The new directory {SAMPLE_DIR} has been created')

NC = 3 # num channels
NZ = 100 # num latent variables
NGF = 64 # num generator filters
NDF = 64 # num discriminator filters
NGPU = 1 # num GPUs
LR = 2e-4 # learning rate
BETA1 = 0.5 # beta1 for Adam optimizer

# pretrained_gen_model_path = os.path.join(MODEL_DIR, f"dcgan_gen_{DATASET}_z100_ep25.pth")
# pretrained_dis_model_path = os.path.join(MODEL_DIR, f"dcgan_dis_{DATASET}_z100_ep25.pth")
pretrained_gen_model_path = None
pretrained_dis_model_path = None

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"



In [5]:
!nvidia-smi

Thu May  4 05:58:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Load Training and Test Dataset

In [6]:
# Transform to (-1, 1) 
img_transform = T.Compose([
    T.Resize(IMAGE_SIZE),
    T.CenterCrop(IMAGE_SIZE),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

if DATASET == 'cifar10':
  dataset_func = datasets.CIFAR10
elif DATASET == 'cifar100':
  dataset_func = datasets.CIFAR100
elif DATASET == 'celebA':
  dataset_func = datasets.CelebA
elif DATASET == 'svhn':
  dataset_func = datasets.SVHN

if DATASET in ['cifar10', 'cifar100']:
  # Load train
  train_data = dataset_func(
      root=DATA_DIR,
      train=True,
      download=True,
      transform=img_transform,
  )

  # Load test
  test_data = dataset_func(
      root=DATA_DIR,
      train=False,
      download=True,
      transform=img_transform,
  )
else:
  # Load train
  train_data = dataset_func(
      root=DATA_DIR,
      split='train',
      download=True,
      transform=img_transform,
  )

  # Load test
  test_data = dataset_func(
      root=DATA_DIR,
      split='test',
      download=True,
      transform=img_transform,
  )

Using downloaded and verified file: /content/drive/My Drive/Colab Notebooks/data/train_32x32.mat
Using downloaded and verified file: /content/drive/My Drive/Colab Notebooks/data/test_32x32.mat


In [7]:
# Create data loader
train_loader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

test_loader = DataLoader(
    test_data,
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

### Define Generator and Discriminator Networks

In [8]:
# Define model architecture and loss function

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.zeros_(m.bias.data)

class Generator(nn.Module):
    def __init__(self, nz, ngf, nc, ngpu=0):
        super().__init__()

        self.ngpu = ngpu

        self.network = nn.Sequential(
            # Input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(inplace=True),

            # state size: (ngf * 8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(inplace=True),

            # state size: (ngf * 4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(inplace=True),

            # state size: (ngf * 2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(inplace=True),

            # state size: (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size: (nc) x 64 x 64
        )
    
    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.network, input, range(self.ngpu))
        else:
            output = self.network(input)
        return output

In [9]:
netG = Generator(NZ, NGF, NC, ngpu=NGPU).to(DEVICE)
netG.apply(weights_init)

if pretrained_gen_model_path is not None:
  netG.load_state_dict(torch.load(pretrained_gen_model_path))
print(netG)

Generator(
  (network): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )

In [10]:
class Discriminator(nn.Module):
    def __init__(self, nz, ndf, nc, ngpu=0):
        super().__init__()
        self.ngpu = ngpu
        self.network = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # state size: (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # state size: (ndf * 2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # state size: (ndf * 4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),

            # state size: (ndf * 8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.network, input, range(self.ngpu))
        else:
            output = self.network(input)
        return output.view(-1, 1).squeeze(1)        

In [11]:
netD = Discriminator(NZ, NDF, NC, ngpu=NGPU).to(DEVICE)
netD.apply(weights_init)
if pretrained_dis_model_path is not None:
  netD.load_state_dict(torch.load(pretrained_dis_model_path))
print(netD)

Discriminator(
  (network): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


In [12]:
# Define loss function and optimizers
criterion = nn.BCELoss()

fixed_noise = torch.randn(64, NZ, 1, 1, device=DEVICE)
real_label = 1
fake_label = 0

optimizerD = optim.Adam(netD.parameters(), lr=LR, betas=(BETA1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=LR, betas=(BETA1, 0.999))

### Train and Store the Models

In [ ]:
gname = "dcgan_gen" 
gen_path = os.path.join(MODEL_DIR, f"{gname}_{DATASET}_z{NZ}_ep{NUM_EPOCH}_{DAY}.pth")

dname = "dcgan_dis" 
dis_path = os.path.join(MODEL_DIR, f"{dname}_{DATASET}_z{NZ}_ep{NUM_EPOCH}_{DAY}.pth")


for epoch in range(NUM_EPOCH):
    
    for batch_idx, (X, _) in enumerate(train_loader):
        start_t = timer.time()
        ############################
        #  Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################

        # Train with real image
        netD.zero_grad()
        real_cpu = X.to(DEVICE)
        batch_size = real_cpu.size(0)
        label = torch.full(
            (batch_size,), real_label,
            dtype=real_cpu.dtype, 
            device=DEVICE
        )

        output = netD(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # Train with fake image
        noise = torch.randn(batch_size, NZ, 1, 1, device=DEVICE)
        fake = netG(noise)
        label.fill_(fake_label)
        output = netD(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # Update G network: minimize log(D(x)) + log(1 - D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label) # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        elapsed_t = timer.time() - start_t

        if batch_idx % 100 == 0:
            print(f'[{epoch+1}/{NUM_EPOCH}] [{batch_idx} / {len(train_loader)}] Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f} D(x): {D_x:.4f} D(G(z)): {D_G_z1:.4f} / {D_G_z2:.4f} Elapsed: {elapsed_t:.2f} secs')
            vutils.save_image(real_cpu, f'{SAMPLE_DIR}/real_samples.png', normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(), f'{SAMPLE_DIR}/fake_samples_epoch-{epoch+1}_batch-{batch_idx}.png', normalize=True)

            # Checkpointing
            torch.save(netD.state_dict(), dis_path)
            print(f" --- Discriminator model stored in {dis_path} ---")

            torch.save(netG.state_dict(), gen_path)
            print(f" --- Generator model stored in {gen_path} ---")        
    # end for batch
# end for epoch

[1/25] [0 / 1145] Loss_D: 1.8308 Loss_G: 6.4696 D(x): 0.5902 D(G(z)): 0.6342 / 0.0022 Elapsed: 2.46 secs
 --- Discriminator model stored in /content/drive/My Drive/Colab Notebooks/models/dcgan_dis_svhn_z100_ep25_4may.pth ---
 --- Generator model stored in /content/drive/My Drive/Colab Notebooks/models/dcgan_gen_svhn_z100_ep25_4may.pth ---
[1/25] [100 / 1145] Loss_D: 0.0031 Loss_G: 40.0834 D(x): 0.9970 D(G(z)): 0.0000 / 0.0000 Elapsed: 0.07 secs
 --- Discriminator model stored in /content/drive/My Drive/Colab Notebooks/models/dcgan_dis_svhn_z100_ep25_4may.pth ---
 --- Generator model stored in /content/drive/My Drive/Colab Notebooks/models/dcgan_gen_svhn_z100_ep25_4may.pth ---
[1/25] [200 / 1145] Loss_D: 0.0006 Loss_G: 39.4786 D(x): 0.9994 D(G(z)): 0.0000 / 0.0000 Elapsed: 0.07 secs
 --- Discriminator model stored in /content/drive/My Drive/Colab Notebooks/models/dcgan_dis_svhn_z100_ep25_4may.pth ---
 --- Generator model stored in /content/drive/My Drive/Colab Notebooks/models/dcgan_gen